In [1]:
import sys 
sys.path.insert(1,"../src/")
sys.path.insert(1,"../src/finn/")
sys.path.insert(1,"../src/qonnx/")

# Create dummy inputs and store in the correct model directory 
For the moment want to create dummy input data, and keep this constant such that it does not change. Use this data to verify that the hw + cpu run has the same output as the expected output.


In [2]:
import numpy as np
import os
import shutil

clear_data = False

# Create a directory with relative info
model_dict = {}

# Get the onnx model directory
md = "../models"
op_type = "Conv"

# Get the model hw directory where the driver file is located.
build_dir = "../build-KV260/"

# Get split names for created hw files
split_dirs = [s for s in os.listdir(os.path.join(build_dir,op_type))]

# Fill the model dict
for split_name in split_dirs:
    hw_base_folder = os.path.join(build_dir,op_type,split_name,"deploy")
    model_base_folder = os.path.join(md,op_type,split_name)
    if clear_data:
        shutil.rmtree(f"{model_base_folder}/data")
    ! ls {model_base_folder}
    model_dict[split_name.replace('_out0','')] = {"model_dir": model_base_folder, "hw_dir": hw_base_folder}


data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx


##### Eventually want to supply a image and see the output

In [3]:
# Create dummy data for the moment
for k,v in model_dict.items():
    sd = v["model_dir"] + "/data"
    if not os.path.exists(sd):
        os.mkdir(sd)
        x = np.random.random([1,3,224,224])
        np.save(f"{sd}/{k}_input.npy",x)
        v["data_dir"] = sd
    else:
        v["data_dir"] = sd

# Get golden output 
For each of the inputs get the expected output from the baseline model

In [4]:
from qonnx.core.modelwrapper import ModelWrapper
from onnxruntime_extensions import get_library_path
import onnxruntime as ort
from custom_ort_functions import set_multithreshold_default
# TODO: ADD THIS STEP TO THE CUSTOM_ORT_FUNCTIONS
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat

# Load input streamlined model
mf = "../models/mobilenet_streamline.onnx"
model = ModelWrapper(mf)
model = model.transform(DoubleToSingleFloat())
set_multithreshold_default(model,mf.replace('.onnx',"_ort.onnx"))

# Set up a session for 
so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())
sess = ort.InferenceSession(mf.replace('.onnx',"_ort.onnx"), so)
print(sess.get_inputs()[0].shape)

for k,v in model_dict.items():
    data_dir = v["data_dir"]
    x = np.load(f"{data_dir}/{k}_input.npy")
    inp_dict = {sess.get_inputs()[0].name: x.astype(np.float32)}
    _gld_res = sess.run([],inp_dict)
    print(_gld_res)
    np.save(f"{data_dir}/{k}_gld_output.npy",_gld_res)

[1, 3, 224, 224]
[array([[644, 818, 530, 111, 626]], dtype=int64)]
[array([[644, 818, 530, 111, 626]], dtype=int64)]
[array([[644, 818, 530, 111, 626]], dtype=int64)]
[array([[644, 818, 530, 111, 626]], dtype=int64)]
[array([[644, 818, 530, 111, 626]], dtype=int64)]
[array([[644, 818, 530, 111, 626]], dtype=int64)]


# Perform the hardware run for the dummy data

In [10]:
from qonnx.core.datatype import DataType
from driver_base import FINNExampleOverlay

io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : [DataType['UINT8']],
    "odt" : [DataType['INT24']],
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : [(1, 224, 224, 3)],
    "oshape_normal" : [(1, 111, 111, 32)],
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : [(1, 224, 224, 3, 1)],
    "oshape_folded" : [(1, 111, 111, 2, 16)],
    "ishape_packed" : [(1, 224, 224, 3, 1)],
    "oshape_packed" : [(1, 111, 111, 2, 48)],
    "input_dma_name" : ['idma0'],
    "output_dma_name" : ['odma0'],
    "number_of_external_weights": 0,
    "num_inputs" : 1,
    "num_outputs" : 1,
}


def exe_mode(batch_size = 1,
    bitfile = "../bitfile/finn-accel.bit",
    inputfile = "input.npy",
    outputfile = "output.npy",
    runtime_weight_dir = "runtime_weights/"
            ):
    
        platform = "zynq-iodma"
        accel = FINNExampleOverlay(
        bitfile_name = bitfile, platform = platform,
        io_shape_dict = io_shape_dict, batch_size = batch_size,
        runtime_weight_dir = runtime_weight_dir
        )
     # load desired input .npy file(s)
        ibuf_normal = []
        for ifn in [inputfile]:
            ibuf_normal.append(np.load(ifn))
        obuf_normal = accel.execute(ibuf_normal)
        if not isinstance(obuf_normal, list):
            obuf_normal = [obuf_normal]
        for o, obuf in enumerate(obuf_normal):
            np.save(outputfile, obuf)

In [15]:
exe = True
for k,v in model_dict.items():
    print("-"*20,k,"-"*20)
    hw_dir = v["hw_dir"]   
    data_dir = v["data_dir"]
    # Load and reshape the input
    x = np.load(f"{data_dir}/{k}_input.npy")
    x = x.reshape(io_shape_dict["ishape_normal"][0])
    x = x.astype(np.uint8)
    # Save the reshaped data for hw input
    np.save(f"{data_dir}/{k}_hw_input.npy",x)
    # execute hw run 
    print(f"{data_dir}/{k}_hw_output.npy")
    if os.path.exists(f"{data_dir}"):
        for f in os.listdir(f"{data_dir}"):
            if f"{k}_hw_output.npy" == f:
                exe = False
    if exe:
        exe_mode(bitfile=f"{hw_dir}/bitfile/finn-accel.bit",
                 inputfile=f"{data_dir}/{k}_hw_input.npy",
                 outputfile=f"{data_dir}/{k}_hw_output.npy")

-------------------- Conv_6 --------------------
../models/Conv/Conv_6_out0/data/Conv_6_hw_output.npy
-------------------- Conv_10 --------------------
../models/Conv/Conv_10_out0/data/Conv_10_hw_output.npy
-------------------- Conv_0 --------------------
../models/Conv/Conv_0_out0/data/Conv_0_hw_output.npy
-------------------- Conv_2 --------------------
../models/Conv/Conv_2_out0/data/Conv_2_hw_output.npy
-------------------- Conv_4 --------------------
../models/Conv/Conv_4_out0/data/Conv_4_hw_output.npy
-------------------- Conv_8 --------------------
../models/Conv/Conv_8_out0/data/Conv_8_hw_output.npy
